In [ ]:
from decodes.core import *
from decodes.io.jupyter_out import JupyterOut
import math

out = JupyterOut.unit_square( )

# A Parametric Representation
todo

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.11.P01.jpg" style="width: 600px; display: inline;">

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.11.P03.jpg" style="width: 200px; display: inline;">

## Curve Objects in Decod.es

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/3.00.D72 Curve Large.jpg" style="width: 800px; display: inline;">

### Visualizing Curves

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.11.P04.jpg" style="width: 200px; display: inline;">

# Features and Idiosyncrasies

### Unequal Division

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.11.P02.jpg" style="width: 200px; display: inline;">

### Non-Unique Parameterizations

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.11.P05.jpg" style="width: 600px; display: inline;">

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.11.P06.jpg" style="width: 200px; display: inline;">

## Composing with Parametric Curves